In [ ]:
import os # for path mainly
import sys # for path mainly
import logging # for status and log output
import matplotlib.pyplot as plt # for plotting charts
import seaborn as sns # for plotting default nice charts
sys.path.append(os.path.abspath("pylib")) # to make helper methods available

from handle_sqlite import save_dataframe_to_db, read_table_as_dataframe

---
load data from dhw

In [ ]:
# load from dwh the newspaper informations per date and their usage of klima
metadata = read_table_as_dataframe("newspapers", "data_output/dwh_data.db")
metadata.head(2)

In [ ]:
# load from dwh the found klima words
context = read_table_as_dataframe("context", "data_output/dwh_data.db")
context.head(2)

---
cast types can help analysis and performance

In [ ]:
context = context.astype({'pre_context': 'string',
                         'post_context': 'string',
                         'prefix': 'string',
                         'suffix': 'string',})
context.info()

In [ ]:
context = context.astype({'pre_context': 'string',
                         'post_context': 'string',
                         'prefix': 'string',
                         'suffix': 'string',})
context.info()

In [ ]:
metadata[metadata.newspaper_name == 'heise']

In [ ]:
# dublicated newspapers published
metadata.duplicated().sum()

In [ ]:
# dublicates of context, including newspaper id. indicates static use
# of klima, like navigation term of a paper.
context.duplicated().sum()

In [ ]:
# % of news pages (not newspaper-companys) with at least once 
# klima in actual data
context.newspaper_id.nunique() / len(context) * 100

% of newspaper-companys, that never mentioned klima

In [ ]:
#metadata.groupby(['newspaper_name'])['klima_mentions_count'].sum()
klima_counts_per_company = metadata.pivot_table(values='klima_mentions_count', index='newspaper_name', aggfunc='sum')

In [ ]:
total_companys = len(klima_counts_per_company)
total_companys

In [ ]:
no_klima_companies = (klima_counts_per_company['klima_mentions_count'] == 0).sum()
no_klima_companies

In [ ]:
no_klima_companies / total_companys * 100

In [ ]:
dict(klima_counts_per_company_sorted)

In [ ]:
# histogram of the times, klima is used in one single paper publish
ax = sns.countplot(data=metadata, x="klima_mentions_count", palette="flare")
#ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
# look at the lower freq edges

top_three = sorted(metadata['klima_mentions_count'].unique())[:5]
counts = metadata['klima_mentions_count'].value_counts().loc[top_three]

ax = sns.barplot(y=counts.index, x=counts.values, orient='h', palette="flare")
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
# look at the higher freq edges and its newspapers

top_three = sorted(metadata['klima_mentions_count'].unique())[-5:]
counts = metadata['klima_mentions_count'].value_counts().loc[top_three]
counts

In [ ]:
ax=sns.countplot(data=context, x="newspaper_id", order=context.newspaper_id.value_counts().iloc[:5].index, palette="flare")
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
ax=sns.countplot(data=context, x="newspaper_id", order=context.newspaper_id.value_counts().iloc[:15].index, palette="flare")
#ax.bar_label(ax.containers[0])
plt.show()

## Merging for deeper eda


In [ ]:
# make sure the one to many relationship meets expectations

In [ ]:
# Count the number of rows for each newspaper_id in the metadata and context tables
metadata_counts = metadata['newspaper_id'].value_counts()

# Check if there are any newspaper_ids that appear more than once in the metadata table
problematic_metadata = metadata_counts[metadata_counts > 1]

print(f"Problematic newspaper_id in metadata (appears more than once):\n{problematic_metadata}")

# Check for any cases where there are no rows in metadata for context's newspaper_id
missing_metadata = context[~context['newspaper_id'].isin(metadata['newspaper_id'])]
print(f"Context rows with missing metadata:\n{missing_metadata}")

In [ ]:
context[~context['newspaper_id'].isin(metadata['newspaper_id'])]

In [ ]:
# Checking the first few rows of both dataframes
print(metadata.head(2))
print(context.head(2))

In [ ]:
import pandas as pd

In [ ]:
# Check for one-to-many relationship based on 'newspaper_id'
merged = pd.merge(context, metadata, on="newspaper_id", how="inner")

# Checking the number of unique newspaper_id in both tables
print(f"Unique newspaper_id in metadata: {metadata['newspaper_id'].nunique()}")
print(f"Unique newspaper_id in context: {context['newspaper_id'].nunique()}")
print(f"Unique newspaper_id in merged: {merged['newspaper_id'].nunique()}")

In [ ]:
merged